# APIs
This week we'll be discussing how to get and process web from the web with Application Programming Interfaces.

In [ ]:
# This is old
import requests
import json
import matplotlib.pyplot as plt
% matplotlib inline
from datetime import datetime


# These are new
import tweepy

# Load secret keys
with open('api_keys.json') as f:
    api_keys = json.loads(f.read())

## Example 1: Bitcoin
In this example, we'll go through the process of manually querying a Bitcoin API
(You can find the API documentation [here](https://blockchain.info/api))

In [ ]:
# Getting exchange rates
# (A simple API with no necessary parameters)
r = requests.get('https://blockchain.info/ticker')
print('status code: ' + str(r.status_code))
r.json()

In [ ]:
# A more complicated API request
# We can use a parameters dictionary to specify 'arguments' to the API
params = {
    'timespan': '1years',
    'rollingAverage': '1days'
}
r = requests.get('https://api.blockchain.info/charts/estimated-transaction-volume-usd', params=params)
print r.url
data = r.json()
data

In [ ]:
x_values = [datetime.fromtimestamp(t['x']) for t in data['values']]
y_values = [t['y'] for t in data['values']]

fig, ax = plt.subplots()
ax.plot(x_values, y_values)
fig.autofmt_xdate()


# Challenges!